# Objective : Identify topics or tags/labels from news articles and map to prospective sales vertical in a prescriptive way.

Author : 
Praveen Vijayan ,
TCS A&I - Pune

# Importing libraries

In [1]:
import csv
import re
import codecs
import os
import spacy
from __future__ import unicode_literals
import pandas as pd
import itertools as it
from gensim.models import Phrases
from gensim.models.word2vec import LineSentence


C:\Users\praveen\Anaconda3\envs\TCS\lib\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.


In [3]:
from __future__ import print_function
import numpy as np
import sys
from time import time
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
import string
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import TruncatedSVD

In [4]:
nlp = spacy.load('en_core_web_sm')

In [5]:
# read CSV file & load into list
with open("IRC_News_data.csv",'r') as my_file:  #,encoding="utf8"
    reader = csv.reader(my_file, delimiter=',')
    news_data_train = list(reader)

# Exploring the data structure

In [6]:
print(news_data_train[3])

['1', 'Pegasystems Earns Best in Biz Award for Second Consecutive Year', 'CAMBRIDGE, Mass., Nov. 30, 2017 (GLOBE NEWSWIRE) -- Pegasystems Inc. (NASDAQ:PEGA), the software company empowering customer engagement at the world’s leading enterprises, today announced that Pega® Intelligent Virtual Assistant has been named a Bronze winner for Best New Product of the Year in the Enterprise category of the seventh annual Best in Biz Awards. This marks the second consecutive year that a Pega solution has won an award in this category.\n\nThe award recognizes Pega Intelligent Virtual Assistant’s use of artificial intelligence (AI) to evolve bots from simple chat and data aggregation novelties into dynamic customer engagement tools. This AI-powered bot provides customers and employees with smart digital assistants from almost any conversational interface – from SMS and email to Facebook, Alexa, Slack, and beyond. Pega Intelligent Virtual Assistant – part of the Pega® Platform – turns these channel

In [7]:
print("class label : " + news_data_train[3][0])

class label : 1


In [8]:
print("Heading : " + news_data_train[3][1])

Heading : Pegasystems Earns Best in Biz Award for Second Consecutive Year


In [9]:
print("Content : " +news_data_train[3][2])

Content : CAMBRIDGE, Mass., Nov. 30, 2017 (GLOBE NEWSWIRE) -- Pegasystems Inc. (NASDAQ:PEGA), the software company empowering customer engagement at the world’s leading enterprises, today announced that Pega® Intelligent Virtual Assistant has been named a Bronze winner for Best New Product of the Year in the Enterprise category of the seventh annual Best in Biz Awards. This marks the second consecutive year that a Pega solution has won an award in this category.

The award recognizes Pega Intelligent Virtual Assistant’s use of artificial intelligence (AI) to evolve bots from simple chat and data aggregation novelties into dynamic customer engagement tools. This AI-powered bot provides customers and employees with smart digital assistants from almost any conversational interface – from SMS and email to Facebook, Alexa, Slack, and beyond. Pega Intelligent Virtual Assistant – part of the Pega® Platform – turns these channels into sophisticated digital agents that make personalized custome

In [10]:
stopwords_file = open("stopwords.txt", "r" , encoding="utf8") 
stopwords = stopwords_file.read()

In [11]:
STOPWORD_customized = {'will','new','york','will' ,'quot','year ','company ','week', 'one','two','three','reuters','reuters' , 'monday','tuesday','wednesday','thursday','friday','saturday','sunday','yesterday'}

In [12]:
#label = [] # class
#text_doc = [] # news text 
text_all = ""
c =0 # to control count of 
for i in news_data_train:
    c= c+1
    if c < 3000 :
        #label.append(i[0])            
        merged = i[1]+" : "+i[2] # combining header and content 
        lowers = merged.lower()   # to lower      
        no_punctuation = re.sub(r'[^\w\s]',' ', lowers)    # remove punctuation
        temp = ' '.join(word for word in no_punctuation.split() if len(word)>3) # remove samll words
        filtered = ' '.join(word for word in temp.split() if word not in stopwords.split()) # remove stopwords
        filtered = ' '.join(word for word in filtered.split() if word not in STOPWORD_customized) # remove stopwords
        temp = filtered
        #text_doc.append(temp)
        text_all = text_all + temp + ". " 

In [13]:
del news_data_train

In [14]:
%%time
parsed_review = nlp(text_all)

Wall time: 27.8 s


In [16]:
#parsed_review

In [16]:
del text_all

# Entity identification 

In [72]:
Entity_list = ""
for num, entity in enumerate(parsed_review.ents):
    #print('Entity {}:'.format(num + 1), entity, '-', entity.label_)
    #print('')
    if entity.label_ == 'GPE' or entity.label_ == 'NORP' or entity.label_ == 'CARDINAL' :
        Entity_list = str(Entity_list) + str(entity) + " : " + entity.label_ + "; "

In [73]:
#review_txt_filepath = "IRC_News_data.csv"

In [74]:
Entity_list.split(';')

['1000 : CARDINAL',
 ' 1000 : CARDINAL',
 ' malaysia : GPE',
 ' american : NORP',
 ' american : NORP',
 ' hundreds : CARDINAL',
 ' half : CARDINAL',
 ' approximately million : CARDINAL',
 ' f2018 : CARDINAL',
 ' 1000 : CARDINAL',
 ' million : CARDINAL',
 ' 2018prophix : CARDINAL',
 ' 2019 : CARDINAL',
 ' bi360 : CARDINAL',
 ' bi360 : CARDINAL',
 ' bi360 : CARDINAL',
 ' 3rdparty : CARDINAL',
 ' eur4 million : CARDINAL',
 ' ucrjaher4rhnwlpnzo5dastg : GPE',
 ' million : CARDINAL',
 ' 12000 : CARDINAL',
 ' 12000 : CARDINAL',
 ' 12000 : CARDINAL',
 ' 12000 : CARDINAL',
 ' half : CARDINAL',
 ' 2022 : CARDINAL',
 ' 2025 : CARDINAL',
 ' 359mhz : CARDINAL',
 ' half : CARDINAL',
 ' half : CARDINAL',
 ' d7a153c6 : GPE',
 ' dozens : CARDINAL',
 ' 1000 : CARDINAL',
 ' millions : CARDINAL',
 ' c3000 : GPE',
 ' seven : CARDINAL',
 ' half : CARDINAL',
 ' half : CARDINAL',
 ' billion : CARDINAL',
 ' million : CARDINAL',
 ' 2018 : CARDINAL',
 ' india : GPE',
 ' hundreds : CARDINAL',
 ' million : CARDINA

# part_of_speech

In [75]:
token_text = [token.orth_ for token in parsed_review]
token_pos = [token.pos_ for token in parsed_review]

#pd.DataFrame(list(zip(token_text, token_pos)),columns=['token_text', 'part_of_speech'])

In [76]:
token_lemma = [token.lemma_ for token in parsed_review]
token_shape = [token.shape_ for token in parsed_review]

#pd.DataFrame(list(zip(token_text, token_lemma, token_shape)),columns=['token_text', 'token_lemma', 'token_shape'])

In [17]:
# parsed_NLP_text.txt

with open("parsed_NLP_text.txt", "w") as text_file:
    print(parsed_review, file=text_file)

In [18]:
def punct_space(token):
    """
    helper function to eliminate tokens
    that are pure punctuation or whitespace
    """
    
    return token.is_punct or token.is_space 

def line_review(filename):
    """
    generator function to read in reviews from the file
    and un-escape the original line breaks in the text
    """
    
    with codecs.open(filename) as f: #,encoding='utf_8'
        for review in f:
            yield review.replace('\\n', '\n')
            
def lemmatized_sentence_corpus(filename):
    """
    generator function to use spaCy to parse reviews,
    lemmatize the text, and yield sentences
    """
    
    for parsed_review in nlp.pipe(line_review(filename),
                                  batch_size=10000, n_threads=4):
        
        for sent in parsed_review.sents:
            yield u' '.join([token.lemma_ for token in sent
                             if not punct_space(token)])

In [19]:
unigram_sentences_filepath = os.path.join('unigram_sentences_all.txt')

In [20]:
%%time
with codecs.open(unigram_sentences_filepath, 'w', encoding='utf_8') as f:
        for sentence in lemmatized_sentence_corpus('parsed_NLP_text.txt'):
            f.write(sentence + '\n')

Wall time: 28.6 s


In [21]:
unigram_sentences = LineSentence(unigram_sentences_filepath)

In [22]:
bigram_model_filepath = 'bigram_model_all.txt'

In [23]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute modeling yourself.
if 1 == 1:

    bigram_model = Phrases(unigram_sentences)

    bigram_model.save(bigram_model_filepath)
    
# load the finished model from disk
bigram_model = Phrases.load(bigram_model_filepath)

Wall time: 428 ms


In [24]:
bigram_sentences_filepath  = 'bigram_model_all.txt'

In [25]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if 1 == 1:

    with codecs.open(bigram_sentences_filepath, 'w' , encoding= 'utf-8') as f:
        
        for unigram_sentence in unigram_sentences:
            
            bigram_sentence = u' '.join(bigram_model[unigram_sentence])
            
            f.write(bigram_sentence + '\n')

C:\Users\praveen\Anaconda3\envs\TCS\lib\site-packages\gensim\models\phrases.py:316: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


Wall time: 489 ms


In [26]:
bigram_sentences = LineSentence(bigram_sentences_filepath)

In [27]:
trigram_model_filepath = 'trigram_model_all.txt'

In [28]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute modeling yourself.
if 1 == 1:

    trigram_model = Phrases(bigram_sentences)

    trigram_model.save(trigram_model_filepath)
    
# load the finished model from disk
trigram_model = Phrases.load(trigram_model_filepath)

Wall time: 402 ms


In [29]:
trigram_sentences_filepath = 'trigram_sentences_all.txt'

In [30]:
trigram_sentences = LineSentence(trigram_sentences_filepath)

In [31]:
trigram_reviews_filepath = 'trigram_transformed_reviews_all.txt'

# From normal text to trigram  

In [32]:
review_txt_filepath = 'parsed_NLP_text.txt' # "sub_News_data_train.csv"


In [33]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to execute data prep yourself.
if 1 == 1:

    with codecs.open(trigram_reviews_filepath, 'w', encoding='utf_8') as f:
        
        for parsed_review in nlp.pipe(line_review(review_txt_filepath),
                                      batch_size=1000, n_threads=4):
            
            #merged = parsed_review # combining header and content 
            #lowers = merged.lower()   # to lower      
            #no_punctuation = re.sub(r'[^\w\s]',' ', lowers)    # remove punctuation
            #temp = ' '.join(word for word in no_punctuation.split() if len(word)>3) # remove samll words
            
            # lemmatize the text, removing punctuation and whitespace
            unigram_review = [token.lemma_ for token in parsed_review
                              if not punct_space(token)]
            
            # apply the first-order and second-order phrase models
            bigram_review = bigram_model[unigram_review]
            trigram_review = trigram_model[bigram_review]
            
            # remove any remaining stopwords
            trigram_review = [term for term in trigram_review
                              if term not in nlp.Defaults.stop_words]
            
            
            
            # write the transformed review as a line in the new file
            trigram_review = u' '.join(trigram_review)
            f.write(trigram_review + '\n')

C:\Users\praveen\Anaconda3\envs\TCS\lib\site-packages\gensim\models\phrases.py:316: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


Wall time: 28.6 s


In [34]:
from gensim.corpora import Dictionary, MmCorpus
from gensim.models.ldamulticore import LdaMulticore

import pyLDAvis
import pyLDAvis.gensim
import warnings
#import cPickle as pickle

In [35]:
trigram_dictionary_filepath = 'trigram_dict_all.dict'

In [36]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to learn the dictionary yourself.
if 1 == 1:

    trigram_reviews = LineSentence(trigram_reviews_filepath)

    # learn the dictionary by iterating over all of the reviews
    trigram_dictionary = Dictionary(trigram_reviews)
    
    # filter tokens that are very rare or too common from
    # the dictionary (filter_extremes) and reassign integer ids (compactify)
    #trigram_dictionary.filter_extremes(no_below=10, no_above=0.4)
    trigram_dictionary.compactify()

    trigram_dictionary.save(trigram_dictionary_filepath)
    
# load the finished dictionary from disk
trigram_dictionary = Dictionary.load(trigram_dictionary_filepath)

Wall time: 102 ms


In [37]:
trigram_bow_filepath = 'trigram_bow_corpus_all.mm'

In [38]:
def trigram_bow_generator(filepath):
    """
    generator function to read reviews from a file
    and yield a bag-of-words representation
    """
    
    for review in LineSentence(filepath):
        yield trigram_dictionary.doc2bow(review)

In [39]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to build the bag-of-words corpus yourself.
if 1 == 1:

    # generate bag-of-words representations for
    # all reviews and save them as a matrix
    MmCorpus.serialize(trigram_bow_filepath,
                       trigram_bow_generator(trigram_reviews_filepath))
    
# load the finished bag-of-words corpus from disk
trigram_bow_corpus = MmCorpus(trigram_bow_filepath)

Wall time: 157 ms


In [40]:
lda_model_filepath = 'lda_model_all.txt'

In [109]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to train the LDA model yourself.
if 1 == 1:

    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        
        # workers => sets the parallelism, and should be
        # set to your number of physical cores minus one
        lda = LdaMulticore(trigram_bow_corpus,
                           num_topics=5,
                           id2word=trigram_dictionary,   # trigram_dictionary
                           workers=4)
    
    lda.save(lda_model_filepath)
    
# load the finished LDA model from disk
lda = LdaMulticore.load(lda_model_filepath)

Wall time: 59.7 s


In [110]:
def explore_topic(topic_number, topn=10):
    """
    accept a user-supplied topic number and
    print out a formatted list of the top terms
    """
        
    print(u'{:20} {}'.format(u'term', u'frequency') + u'\n')

    for term, frequency in lda.show_topic(topic_number, topn=25):
        print(u'{:20} {:.3f}'.format(term, round(frequency, 3)))

In [111]:
#explore_topic(topic_number=3)

In [112]:
LDAvis_data_filepath = 'ldavis_prepared.txt'

In [113]:
%%time
LDAvis_prepared = pyLDAvis.gensim.prepare(lda, trigram_bow_corpus,trigram_dictionary)

C:\Users\praveen\Anaconda3\envs\TCS\lib\site-packages\pyLDAvis\_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


Wall time: 10.9 s


In [114]:
pyLDAvis.display(LDAvis_prepared)

# Topic of a given text 

In [48]:
def lda_description(review_text, min_topic_freq=0.05):
    """
    accept the original text of a review and (1) parse it with spaCy,
    (2) apply text pre-proccessing steps, (3) create a bag-of-words
    representation, (4) create an LDA representation, and
    (5) print a sorted list of the top topics in the LDA representation
    """
    
    # parse the review text with spaCy
    parsed_review = nlp(review_text)
    
    # lemmatize the text and remove punctuation and whitespace
    unigram_review = [token.lemma_ for token in parsed_review
                      if not punct_space(token)]
    
    # apply the first-order and secord-order phrase models
    bigram_review = bigram_model[unigram_review]
    trigram_review = trigram_model[bigram_review]
    
    # remove any remaining stopwords
    #trigram_review = [term for term in trigram_review
    #                  if not term in spacy.en.STOPWORDS]
    
    # create a bag-of-words representation
    review_bow = trigram_dictionary.doc2bow(trigram_review)
    
    # create an LDA representation
    review_lda = lda[review_bow]
    
    # sort with the most highly related topics first
    #review_lda = sorted(review_lda, key=lambda (topic_number, freq): -freq)
    
    for topic_number, freq in review_lda:
        if freq < min_topic_freq:
            break
            
        # print the most highly related topic names and frequencies
        print("The text belongs to topic (with probability): ")
        print('{:25} {}'.format(topic_number,
                                round(freq, 3)))

In [ ]:
review_txt_filepath = 

In [49]:
def get_sample_review(review_number):
    """
    retrieve a particular review index
    from the reviews file and return it
    """
    
    return list(it.islice(line_review(review_txt_filepath),
                          review_number, review_number+1))[0]

In [51]:
sample_review = get_sample_review(50)
print(sample_review)

IndexError: list index out of range

In [269]:
lda_description(sample_review)

C:\Users\praveen\Anaconda3\envs\TCS\lib\site-packages\gensim\models\phrases.py:316: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


The text belongs to topic (with probability): 
                        7 0.955


In [77]:
from gensim.models import Word2Vec

trigram_sentences = LineSentence(bigram_sentences_filepath)
word2vec_filepath = 'word2vec_model_all.txt'

In [90]:
%%time

# this is a bit time consuming - make the if statement True
# if you want to train the word2vec model yourself.
if 1 == 1:

    # initiate the model and perform the first epoch of training
    news2vec = Word2Vec(trigram_sentences, size=100, window=10,
                        min_count=5, sg=1, workers=1)
    
    news2vec.save(word2vec_filepath)

    # perform another 11 epochs of training
    for i in range(1,12):

        news2vec.train(trigram_sentences,total_examples=1000,epochs=i)
        news2vec.save(word2vec_filepath)
        
# load the finished model from disk
news2vec = Word2Vec.load(word2vec_filepath)
news2vec.init_sims()

print(u'{} training epochs so far.'.format(news2vec.train_count))

12 training epochs so far.
Wall time: 1min 10s


# Get related words using word2vec

In [91]:
def get_related_terms(token, topn=15):
    """
    look up the topn most similar terms to token
    and print them as a formatted list
    """

    for word, similarity in news2vec.most_similar(positive=[token], topn=topn):

        print(u'{:20} {}'.format(word, round(similarity, 3)))

In [93]:
get_related_terms(u'artificial_intelligence')

scientific           0.424
augmentation         0.423
finance              0.397
buzzword             0.397
tech_giant           0.374
simulation           0.364
fuel                 0.363
machine_learn        0.358
tell                 0.357
bot                  0.356
profession           0.353
occur                0.351
tech                 0.347
cognitive            0.345
register             0.345


In [94]:
get_related_terms(u'blockchain')

blockchain_technology 0.73
information_builder  0.48
bitcoin              0.458
tourism              0.457
cryptocurrency       0.453
botto                0.444
progress             0.442
infosy               0.434
iway                 0.432
organisation         0.429
ledger               0.418
dhaheri              0.417
forum                0.407
dubai                0.406
supply_chain         0.384


In [95]:
get_related_terms(u'cloud')

service              0.53
enterprise           0.528
platform             0.525
acrow                0.496
premise              0.476
customer             0.473
integration          0.466
integrate            0.466
fully                0.463
ease                 0.461
mendix               0.46
microsoft_azure      0.459
saa                  0.445
hana                 0.443
pega                 0.442


In [96]:
get_related_terms(u'google')

triangle             0.585
kubernete            0.481
bigquery             0.472
asic                 0.461
gpus                 0.454
atscale              0.423
dream                0.422
custom               0.413
silicon              0.412
exactly              0.411
division             0.408
comparison           0.4
serverless           0.398
layout               0.392
alliance             0.391


In [97]:
get_related_terms(u'microsoft') 

azure                0.628
micron               0.531
linux                0.524
microsoft_azure      0.439
hannover             0.424
enterprise_explorer  0.423
traveler             0.422
appian               0.41
foundation           0.409
visio                0.405
diagram              0.404
bpmn                 0.4
hyper                0.384
taiwan               0.382
tech_giant           0.382


In [98]:
get_related_terms(u'intel')

telit                0.622
devicewise           0.618
chipmaker            0.497
alibaba              0.446
brian                0.438
jointly              0.435
processor            0.43
joint                0.427
doug                 0.416
gateway              0.412
architecture         0.395
breakthrough         0.392
internet_thing       0.384
kubernete            0.38
heat                 0.372


In [99]:
get_related_terms(u'car')

self_drive           0.747
tesla                0.493
connectivity         0.447
automobile           0.423
fly                  0.42
equip                0.415
silicon_valley       0.411
telematic            0.411
appear               0.397
street               0.395
neurala              0.391
cloudflare           0.388
siri                 0.385
smart                0.383
autonomous           0.382


In [100]:
get_related_terms(u'app')

mobilefirst          0.494
garage               0.487
android              0.469
apple                0.429
iphone               0.418
fintech              0.416
unificationengine    0.415
developer            0.414
iqlect               0.41
alike                0.394
proprietary          0.391
device_farm          0.389
progressive          0.388
cio                  0.382
pace                 0.376


In [104]:
get_related_terms(u'digitization')

watson_consult       0.487
opportunity          0.466
political            0.461
help_client          0.453
asian                0.431
pervasive            0.406
diverse              0.4
various              0.396
growth_opportunity   0.394
barrier              0.394
health_care          0.393
consult              0.388
currency             0.388
internet             0.387
accenture_health     0.387


In [105]:
get_related_terms(u'facebook')

snapchat             0.622
youtube              0.468
story                0.464
button               0.453
bolster              0.437
social_medium        0.432
cool                 0.429
whatsapp             0.401
play                 0.397
active               0.395
talent               0.39
exceed               0.389
twitter              0.38
netflix              0.376
prospect             0.375
